In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [2]:
from agents import Agent, Runner
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline: str
    tutorial: str

tutorial_generator = Agent(
    name="Tutorial Generator",
    handoff_description="Used for generating a tutorial based on an outline.",
    instructions=(
        "Given a programming topic and an outline, your job is to generate code snippets for each section of the outline."
        "Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples."
        "Where it makes sense, include comments in the code snippets to further explain the code."
    ),
    output_type=Tutorial
)

outline_builder = Agent(
    name="Outline Builder",
    instructions=(
        "Given a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline."
        "After making the outline, hand it to the tutorial generator agent."
    ),
    handoffs=[tutorial_generator]
)

tutorial_response = await Runner.run(outline_builder, "Loops in Java")
print(tutorial_response.final_output)   

Here's an outline for a tutorial on "Loops in Java":

1. **Introduction**
   - Brief overview of why loops are used in programming.
   - Different types of loops available in Java.
  
2. **For Loop**
   - Structure and syntax of a for loop.
   - Example: Iterating over an array.
   - Common use cases.

3. **Enhanced For Loop (For-Each)**
   - Introduction to the for-each loop.
   - Syntax and structure.
   - Example: Iterating over elements in a collection or array.
   - Benefits and limitations.

4. **While Loop**
   - Introduction and use cases.
   - Syntax and structure.
   - Example: Simple counter.
   - Comparing while loop with for loop.

5. **Do-While Loop**
   - Differences between while and do-while loops.
   - Syntax and structure.
   - Example: A loop that guarantees at least one execution.
   - Use cases where a do-while loop is preferable.

6. **Nested Loops**
   - Explanation of nesting loops.
   - Example: Printing a pattern or iterating over a 2D array.
   - Considerati

In [3]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

def on_history_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to history tutor agent")

# This agent has the capability to handoff to either the history or math tutor agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), 
              handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

In [4]:
result = await Runner.run(triage_agent, "How do I add 2 and 2?")
result.final_output

Handing off to math tutor agent


"Adding 2 and 2 is a simple arithmetic problem. Here's how you do it:\n\n1. **Identify the Numbers**: You have two numbers you want to add: 2 and 2.\n\n2. **Combine the Numbers**: When you add, you are combining these numbers to get a total. Think of it as putting two groups of two objects together.\n\n3. **Calculate the Sum**:\n   - Think of it like counting: Start with 2, then count 2 more. So, 2 + 2 equals 4.\n\n4. **Result**: The sum of 2 and 2 is 4.\n\n### Example\n\nImagine you have 2 apples and your friend gives you 2 more apples. You would then have a total of 4 apples. \n\nThis simple operation is the foundation of addition, and understanding it is crucial for more complex math problems."

In [5]:
result = await Runner.run(triage_agent, "How did WW2 start?")
result.final_output

Handing off to history tutor agent


"World War II began on September 1, 1939, when Germany, led by Adolf Hitler, invaded Poland. This aggressive action directly violated international agreements and prompted Britain and France to declare war on Germany two days later, on September 3, 1939.\n\n### Key Context and Events Leading to the War:\n\n1. **Treaty of Versailles (1919):** After World War I, the Treaty of Versailles imposed heavy reparations and territorial losses on Germany. This caused economic hardship and resentment that fueled nationalist and militaristic sentiments.\n\n2. **Rise of Totalitarian Regimes:** The 1930s saw the rise of totalitarian governments in Germany, Italy, and Japan. In Germany, Adolf Hitler and the Nazi Party capitalized on economic woes and national humiliation to gain power, promoting an aggressive expansionist ideology.\n\n3. **Expansionist Policies:** Hitler pursued aggressive territorial expansion, starting with the reoccupation of the Rhineland in 1936, the annexation of Austria in 1938

In [6]:
from agents import function_tool

class ManagerEscalation(BaseModel):
    issue: str # the issue being escalated
    why: str # why can you not handle it? Used for training in the future

@function_tool
def create_ticket(issue: str):
    """"
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In a real-world scenario, this would interact with a ticketing system

manager_agent = Agent(
    name="Manager",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial custom service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools=[create_ticket],
)

def on_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print("Escalating to manager agent: ", input.issue)
    print("Reason for escalation: ", input.why)

    # here we might store the escalation in a database or log it for future reference

customer_service_agent = Agent(
    name="Customer Service",
    instructions="You assist customers with general inquiries and basic troubleshooting. " +
                 "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself.",
    handoffs=[handoff(
        agent=manager_agent,
        input_type=ManagerEscalation,
        on_handoff=on_manager_handoff,
    )]
)

result = await Runner.run(customer_service_agent, "Hello how much are tickets?")
print(result.final_output)

Could you please provide more details about the event or destination for which you need ticket information?
